In [7]:
# load in csvOut.csv data (holds image name & classification)
import csv
import os #to get all picture in directory
from PIL import Image #to open images
import numpy as np #load in numpy module
import sys
import pickle

# loads in full csvOut data (name & classification)
with open('..//annotations_handheld_limited.csv', newline='') as csvfile:
    cls_full = list(csv.reader(csvfile))

In [8]:
# Get the list of all files in directory
path = "..//images_handheld_resized"
dir_list = os.listdir(path)

# Initialize dictionaries to hold bitmap information for each image
rgb = {}
rgb_list = []
cls = []

# loop through files and get bit map for each (save as object where filename => bitmap for r,g,b)
for index, file in enumerate(cls_full):
    
    # push classification to list
    cls.append(file[1])
    
    # method found https://stackoverflow.com/questions/46385999/transform-an-image-to-a-bitmap
    img = Image.open(path + "\\" + file[0])
    
    # resize image
    small_img = img.resize((600, 400))
    smaller_img = img.resize((300, 200))
    smallest_img = img.resize((120, 80))
    ary = np.array(smallest_img)
    
    # Save all 3 together
    rgb[file[0]] = ary
    
    # Split the three channels
    #r[file], g[file], b[file] = np.split(ary,3,axis=2)
    
    # flatten out to 1 array
    rgb[file[0]]=rgb[file[0]].reshape(-1)
    rgb_list.append(rgb[file[0]])
    
    # convert array to string
    #rgb_string[file] = np.array2string(rgb[file], precision=2, separator=',', suppress_small=True)


In [9]:
# convert bitmaps to indexed np array
rgb_arr = np.array([rgb[row] for row in rgb])
rgb_arr2 = np.array(rgb_list)

# convert classifications to numpy array
cls_np = np.array(cls)

In [10]:
# Training model (load necessary libraries)
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(rgb_arr2, cls_np, test_size=0.2)

In [ ]:
# Scale data to make it easier for ML model to process
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)

In [ ]:
# https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
# use gridsearchcv to optimize hyperparameters    
clf = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

# Train SVM Model
clf.fit(X_train, y_train.ravel())

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print("Accuracy: ", accuracy)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.901 total time= 2.6min
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.902 total time= 2.9min


In [16]:
# https://medium.com/analytics-vidhya/image-classification-using-machine-learning-support-vector-machine-svm-dc7a0ec92e01
# defining parameter range
param_grid={
    'C':[0.1],
    'gamma':[0.0001],
    'kernel':['sigmoid','rbf', 'poly', 'linear']
}

# Initialize SVC model & pass param grid options to gridsearchcv
clf = GridSearchCV(svm.SVC(probability=True),param_grid, refit = True, verbose = 3)
#clf = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3) # possible 2nd options from other research 

# Train SVM Model
clf.fit(X_train, y_train.ravel())

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print("Accuracy: ", accuracy)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END C=0.1, gamma=0.0001, kernel=sigmoid;, score=0.574 total time=  54.2s
[CV 2/5] END C=0.1, gamma=0.0001, kernel=sigmoid;, score=0.574 total time=  57.9s
[CV 3/5] END C=0.1, gamma=0.0001, kernel=sigmoid;, score=0.570 total time=  54.0s
[CV 4/5] END C=0.1, gamma=0.0001, kernel=sigmoid;, score=0.572 total time= 1.0min
[CV 5/5] END C=0.1, gamma=0.0001, kernel=sigmoid;, score=0.572 total time= 1.1min
[CV 1/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.574 total time= 1.7min
[CV 2/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.574 total time= 1.8min
[CV 3/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.570 total time= 2.1min
[CV 4/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.572 total time= 1.8min
[CV 5/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.572 total time= 1.8min
[CV 1/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.578 total time= 1.2min
[CV 2/5] END ..C=0.1, gamma=0.0001, kernel=p

In [22]:
print(clf.best_estimator_)

SVC(C=0.1, gamma=0.0001, kernel='sigmoid', probability=True)


In [11]:
filename = '../app/pre_trained_models/best_trained1.sav'
pickle.dump(clf, open(filename, 'wb'))

In [7]:
# https://medium.com/analytics-vidhya/image-classification-using-machine-learning-support-vector-machine-svm-dc7a0ec92e01
# defining parameter range
param_grid = {
    'C': [0.1, 1, 10],              # regularization parameter
    'degree': [2, 3, 4],            # degree of the polynomial kernel
    'coef0': [0, 1, 2],             # independent term in kernel function
    'gamma': ['scale', 'auto'],     # kernel coefficient
    'kernel': ['poly']
}

# Initialize SVC model & pass param grid options to gridsearchcv
clf = GridSearchCV(svm.SVC(probability=True),param_grid, refit = True, verbose = 3)
#clf = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3) # possible 2nd options from other research 

# Train SVM Model
clf.fit(X_train, y_train.ravel())

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print("Accuracy: ", accuracy)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5] END C=0.1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.944 total time=   4.5s
[CV 2/5] END C=0.1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.944 total time=   4.5s
[CV 3/5] END C=0.1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.944 total time=   4.7s
[CV 4/5] END C=0.1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.938 total time=   6.0s
[CV 5/5] END C=0.1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.938 total time=   4.9s
[CV 1/5] END C=0.1, coef0=0, degree=2, gamma=auto, kernel=poly;, score=0.950 total time=   4.8s
[CV 2/5] END C=0.1, coef0=0, degree=2, gamma=auto, kernel=poly;, score=0.956 total time=   5.0s
[CV 3/5] END C=0.1, coef0=0, degree=2, gamma=auto, kernel=poly;, score=0.950 total time=   5.1s
[CV 4/5] END C=0.1, coef0=0, degree=2, gamma=auto, kernel=poly;, score=0.944 total time=   4.9s
[CV 5/5] END C=0.1, coef0=0, degree=2, gamma=auto, kernel=poly;, scor

[CV 1/5] END C=0.1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   4.8s
[CV 2/5] END C=0.1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.963 total time=   5.2s
[CV 3/5] END C=0.1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   4.8s
[CV 4/5] END C=0.1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.963 total time=   5.0s
[CV 5/5] END C=0.1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   4.9s
[CV 1/5] END C=1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.950 total time=   4.9s
[CV 2/5] END C=1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.956 total time=   4.9s
[CV 3/5] END C=1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.950 total time=   5.7s
[CV 4/5] END C=1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.944 total time=   5.8s
[CV 5/5] END C=1, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.950 total time=   5.2s
[CV 1/5] END C=1, coef0=0, degree=2, gamma=au

[CV 3/5] END C=1, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.950 total time=   8.0s
[CV 4/5] END C=1, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.963 total time=   6.4s
[CV 5/5] END C=1, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.950 total time=   5.3s
[CV 1/5] END C=1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   5.2s
[CV 2/5] END C=1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.963 total time=   5.5s
[CV 3/5] END C=1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   6.6s
[CV 4/5] END C=1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.963 total time=   5.7s
[CV 5/5] END C=1, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   5.3s
[CV 1/5] END C=10, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.950 total time=   5.1s
[CV 2/5] END C=10, coef0=0, degree=2, gamma=scale, kernel=poly;, score=0.956 total time=   5.1s
[CV 3/5] END C=10, coef0=0, degree=2, gamma=scale, ke

[CV 4/5] END C=10, coef0=2, degree=3, gamma=auto, kernel=poly;, score=0.963 total time=   5.3s
[CV 5/5] END C=10, coef0=2, degree=3, gamma=auto, kernel=poly;, score=0.950 total time=   5.0s
[CV 1/5] END C=10, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.944 total time=   5.0s
[CV 2/5] END C=10, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.963 total time=   5.1s
[CV 3/5] END C=10, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.950 total time=   4.9s
[CV 4/5] END C=10, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.963 total time=   5.1s
[CV 5/5] END C=10, coef0=2, degree=4, gamma=scale, kernel=poly;, score=0.950 total time=   5.1s
[CV 1/5] END C=10, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   5.2s
[CV 2/5] END C=10, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.963 total time=   5.0s
[CV 3/5] END C=10, coef0=2, degree=4, gamma=auto, kernel=poly;, score=0.944 total time=   6.2s
[CV 4/5] END C=10, coef0=2, degree=4, gamma=a

In [24]:
# https://medium.com/analytics-vidhya/image-classification-using-machine-learning-support-vector-machine-svm-dc7a0ec92e01
# defining parameter range
param_grid = {
    'C': [0.01],      # regularization parameter
    'kernel': ['linear']
}

# Initialize SVC model & pass param grid options to gridsearchcv
clf = GridSearchCV(svm.SVC(probability=True), param_grid, refit = True, verbose = 3, cv=5)
#clf = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3) # possible 2nd options from other research 

# Train SVM Model
clf.fit(X_train, y_train.ravel())

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print("Accuracy: ", accuracy)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END .............C=0.01, kernel=linear;, score=0.605 total time= 1.1min
[CV 2/5] END .............C=0.01, kernel=linear;, score=0.547 total time= 1.1min
[CV 3/5] END .............C=0.01, kernel=linear;, score=0.581 total time= 1.0min
[CV 4/5] END .............C=0.01, kernel=linear;, score=0.588 total time= 1.1min
[CV 5/5] END .............C=0.01, kernel=linear;, score=0.533 total time= 1.2min
Accuracy:  0.5372670807453416


In [26]:
for train_index, test_index in clf.cv.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    print("Training set shape:", X_train.shape, y_train.shape)
    print("Test set shape:", X_test.shape, y_test.shape)

AttributeError: 'int' object has no attribute 'split'

In [27]:
clf

GridSearchCV(cv=5, estimator=SVC(probability=True),
             param_grid={'C': [0.01], 'kernel': ['linear']}, verbose=3)